# Basic preparations

Using ArcHydro phase 2 dataset 

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import hydrant.topology.geom as gm

import subprocess
import os

Path definitions (system dependant - modify accordingly):

In [2]:
# geofabric paths

# output paths
output_path = './results/'

# Reading Geofabric Dataset

Case study of Athabasca River from ArcHydro data set

In [3]:
# file names
cat_files = ['../shapefiles/basins_renamedID.shp']
riv_files = ['../shapefiles/river_with_slope.shp']

cat = pd.concat([gpd.read_file(f) for f in cat_files])
riv = pd.concat([gpd.read_file(f) for f in riv_files])

In [4]:
cat.to_crs(epsg=4326, inplace=True)
riv.to_crs(epsg=4326, inplace=True)

# Preparing `cat`, `riv`, and `nca` objects for AB

## Preparing `catchments` and `rivers`

In [6]:
catchments = gm.prepare_cat(
    cat=cat, 
    cat_col_id='DrainID', 
)


In [7]:
catchments.head()

,DrainID,GridID,NextDownID,JunctionID,Shape_Leng,Shape_Area,geometry,hillslope
0,1500000001,1,-1,1500076622,15600.4142,1.250066e+07,"POLYGON ((-111.01517 58.73518, -111.01527 58.7...",0
1,1500000002,2,-1,1500076625,10800.2866,3.450183e+06,"POLYGON ((-111.06912 58.73234, -111.06983 58.7...",0
2,1500000003,3,-1,1500076617,800.0212,3.000159e+04,"POLYGON ((-110.95588 58.72434, -110.95609 58.7...",0
3,1500000004,4,1500000003,1500076616,12400.3288,5.080269e+06,"POLYGON ((-110.99286 58.73359, -110.99296 58.7...",0
4,1500000005,5,1500000003,1500076616,17400.4618,1.088058e+07,"POLYGON ((-110.95436 58.72249, -110.95447 58.7...",0


In [9]:
rivers = gm.prepare_riv(
    riv=riv, 
    riv_cols={
        'id':'DrainID', 
        'next_id':'NextDrain', 
        'slope':'Slope', 
        'length':'Shape_Leng', 
    }, 
    cat=catchments, 
    cat_cols={
        'id':'DrainID', 
        'hillslope':'hillslope',
        'geom':'geometry'
    }
)


In [10]:
rivers.head()

,GridID,FROM_NODE,TO_NODE,HydroID,NextDownID,DrainID,Shape_Leng,Slope,NextDrain,geometry,hillslope
0,1,6,1,1500017088,-1,1500000001,2527.884607,1.0,-9999,"LINESTRING (-111.02918 58.71853, -111.02644 58...",0
1,2,7,2,1500017089,-1,1500000002,2047.110588,1.0,-9999,"LINESTRING (-111.08047 58.71605, -111.08696 58...",0
2,3,3,4,1500017086,-1,1500000003,70.712587,1.0,-9999,"LINESTRING (-110.95689 58.72302, -110.95598 58...",0
3,4,5,3,1500017087,1500017086,1500000004,291.429104,1.0,1500000003,"LINESTRING (-110.96131 58.72226, -110.95872 58...",0
4,5,16,3,1500017094,1500017086,1500000005,4082.951008,1.0,1500000003,"LINESTRING (-110.9645 58.68728, -110.96267 58....",0


In [11]:
# saving data
try:
    os.makedirs(output_path)
except FileExistsError:
    pass
    
catchments.to_file(os.path.join(output_path, 'basins.shp'))
rivers.to_file(os.path.join(output_path, 'rivers.shp'))
